# [Instructions for this project](https://automatetheboringstuff.com/2e/chapter18/)

I tested this on an inbox with 3 promotional emails I forwarded to it. There should have been 2 unsub links, but one was in a u tag I believe. Still, this could be useful in most cases.

In [1]:
# Practice project: Auto Unsubscriber

# Import libraries
import imapclient, imaplib, pyzmail, bs4, webbrowser, pyinputplus as pyip

# Connect to email
imapObj = imapclient.IMAPClient('imap.gmail.com', ssl=True)

EMAIL = pyip.inputStr(prompt = 'Enter email address: ')
PASSWORD = pyip.inputStr(prompt = 'Enter password: ')

imapObj.login(EMAIL, PASSWORD)

imaplib._MAXLINE = 10000000

# Download emails
imapObj.select_folder('INBOX', readonly=True)
UIDs = imapObj.search(['ALL'])
rawMessages = imapObj.fetch(UIDs, ['BODY[]'])

# Add message HTMLs to a list

messageHtmlList = []
for i in rawMessages:
    message = pyzmail.PyzMessage.factory(rawMessages[i][b'BODY[]'])
    if message.html_part != None:
        messageHtml = message.html_part.get_payload().decode(message.html_part.charset)
        messageHtmlList.append(messageHtml)
        
# Scan for unsubscribe
unSubLinkList = []

for messageHtml in messageHtmlList:
    soup = bs4.BeautifulSoup(messageHtml,'html.parser')
    linkTag = soup.find("a",string = 'unsubscribe')
    
    if linkTag != None:
        unSubLink = linkTag['href']
        unSubLinkList.append(unSubLink)        
        
# Open links
for link in unSubLinkList:
    webbrowser.open(link)